# C14-hepatocyte analysis

In [1]:
# m = c14.models.liver.A()

# path = c14.models.liver.default_path

# from scipy.interpolate import UnivariateSpline

# ploidy_data = pd.read_excel(path)
# ploidy_data['age'] = (ploidy_data[['age_min', 'age_max']].
#                       mean(axis='columns'))
# ploidy_data /= 100.0
# ploidy_data['age'] *= 100

# ploidy = UnivariateSpline(ploidy_data['age'].values,
#                                ploidy_data['2C_mean'].values,
#                                ext=3, k=2)

# t = np.linspace(0, 100)
# plt.plot(t, m.ploidy(t))
# plt.plot(t, ploidy(t))

default_parameters -> fit

default_limit -> physical

parameters -> physical

In [2]:
import datetime
from IPython.display import display
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

%matplotlib inline

mem = joblib.Memory(location='/scratch/tmp/fabrost/hepato', verbose=0)

import logging

%load_ext rpy2.ipython

# logging.basicConfig(filename='c14.log',
#                              filemode='a',
#                              format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
#                             datefmt='%H:%M:%S',
#                              level=logging.DEBUG)
logging.basicConfig(level=logging.WARNING)

In [3]:
logger = logging.getLogger()
logger.setLevel(logging.WARNING)

In [4]:
# %load_ext autoreload
# %autoreload 2
import c14
import c14.models.liver
import c14.models.minimal

## Data

In [5]:
data = pd.read_csv('../data/C14data_liver_samples_20191023.csv')
data = data.groupby(['type', 'sample', 'ploidy', 'pathology']).mean().dropna(how='all').reset_index()
data['age'] = data['Dcoll'] - data['Dbirth']
data = data.query('type == "hepatocyte" and pathology != "Y"')
edata = c14.exp_data(data)

In [6]:
edata_young = c14.exp_data(data.query('age <= 50'))
edata_old = c14.exp_data(data.query('age > 50'))

In [27]:
m = c14.models.liver.D()

In [31]:
p0 = {'kappa24': -2.1454785168675974,
 'kappa42': -2.8893391579364014,
 'r2': -1.1713069980603095,
 'r4': -1.4919629303134156}

In [32]:
op = c14.optimize(m, edata, step_size=0.5)

In [34]:
res = op.optimize_minuit(p0)

In [35]:
vis = c14.visualisze(m, edata, step_size=0.5)

In [37]:
res

({'kappa24': -3.996357468498116,
  'kappa42': -2.784155065601673,
  'r2': -0.3428598339567834,
  'r4': -1.1423588557739561},
 {'kappa24': 1.02349451906055,
  'kappa42': 1.0045602538238187,
  'r2': 0.2512200440163322,
  'r4': 0.12360546631937849},
 -103.8078090536359,
 False,
           kappa24   kappa42        r2        r4
 kappa24  1.000000 -0.089867  0.792451 -0.716253
 kappa42 -0.089867  1.000000 -0.090113  0.066411
 r2       0.792451 -0.090113  1.000000 -0.569654
 r4      -0.716253  0.066411 -0.569654  1.000000,
           kappa24   kappa42        r2        r4
 kappa24  1.047541 -0.092397  0.203757 -0.090613
 kappa42 -0.092397  1.009141 -0.022741  0.008246
 r2       0.203757 -0.022741  0.063112 -0.017689
 r4      -0.090613  0.008246 -0.017689  0.015278)

In [38]:
pe = res[0]

In [39]:
cov = res[-1]

In [40]:
error = vis.calc_error(pe, cov, ignore_physics=True)

In [100]:
values = vis.get_parameter(pe, cov, t_eval=0).loc[0]
values

,point_estimate,lower,upper
parameter,,,
delta2,0.455732,0.447784,1.07523
delta4,0.0636877,0.0495087,0.132823
kappa24,0.000100842,9.58529e-06,0.00106091
kappa42,0.00164378,0.000163198,0.0165567
lambda2,0.454189,0.447116,1.09124
lambda4,0.073695,0.0513032,0.0887965
p2,0.999778,0.995116,0.99966
p4,0.999778,0.995116,0.99966
r2,0.454088,0.254846,0.809101


In [46]:
import scipy as sp

In [74]:
bootstrap_sample = pd.DataFrame(sp.random.multivariate_normal([pe[name] for name in cov.index], cov.values, size=10000), columns=cov.index)
bootstrap_sample.head()

,kappa24,kappa42,r2,r4
0,-2.972342,-2.986293,-0.239625,-1.176183
1,-3.111542,-3.411981,-0.075880,-1.338597
2,-4.156341,-2.907498,-0.409692,-0.984101
3,-2.053095,-3.005943,0.037014,-1.401575
4,-4.800033,-2.146119,-0.738819,-1.077857


In [92]:
(10**bootstrap_sample['r2']).median()

0.45788726378264216

In [78]:
lambda2_sample = 10**bootstrap_sample['r2'] + 10**bootstrap_sample['kappa24']

Hence, the median value for lambda2 and the 68.2% CI should be:

In [98]:
np.percentile(lambda2_sample, [50, 15.9, 100-15.9])

array([0.45822066, 0.25348697, 0.81245811])

However, our bootstrapping gives:

In [105]:
values.loc['lambda2'].astype(float).values

array([0.45418899, 0.4471156 , 1.09124434])